Importar librerias requeridas

In [1]:
import ee 
import geemap
import json
import os

Autenticación con GEE y extracción de zona de estudio

In [10]:
ee.Authenticate()
ee.Initialize(project='ee-rimartinezs')

with open(os.path.join(os.getcwd(), '..', 'data', 'geoBoundaries-COL-ADM0.geojson')) as geojson:
    data = json.load(geojson)

colombia = ee.FeatureCollection(data)

# Cargar el raster de elevación y el vector de Colombia
elevation = ee.Image('USGS/GMTED2010_FULL').select('mea').rename('elevation')


Extracción Poligonos de elevación

In [ ]:

#Elegir intervalo de elvación

zones = elevation.divide(750).floor().toInt().rename('zone')

# Enmascara valores menores o iguales 0
zones = zones.updateMask(elevation.gt(0))

# Recorte Colombia
zones_clipped = zones.clip(colombia)

# Convertir raster a polígonos
elevation_vector = zones_clipped.reduceToVectors(
    geometry=colombia,
    crs=elevation.projection(),
    scale=1000,  
    geometryType='polygon',
    eightConnected=False,
    labelProperty='zone',
    bestEffort=False,
    maxPixels=1e13,
    tileScale=3
)

Mapa

In [ ]:

Map = geemap.Map(center=[4, -74], zoom=6)
Map.addLayer(elevation.clip(colombia), {'min': 0, 'max': 4000}, 'Elevation')
Map.addLayer(zones_clipped, {
    'min': 0,
    'max': 10, 
    'palette': ['white', 'yellow', 'orange', 'green', 'blue', 'purple', 'brown'],
    'opacity': 0.6
}, 'Elevation Zones (800)')
Map.addLayer(elevation_vector, {}, 'Elevation Zones - Vector')
Map.addLayer(colombia, {}, 'Colombia Border')
Map

Map(center=[4, -74], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…